In [1]:
import pandas as pd
from processor.processor import Processor as processor
from database.adatabase import ADatabase
import matplotlib.pyplot as plt
from tqdm import tqdm
from xgboost import XGBRegressor
import warnings
warnings.simplefilter(action="ignore")
import numpy as np
from datetime import datetime
from arch import arch_model
from statsmodels.tsa.arima.model import ARIMA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from statistics import mean

In [2]:
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",attrs={"id":"constituents"})[0].rename(columns={"Symbol":"ticker"})

In [13]:
def calculate_technical_indicators(price):
    # Average Return
    price["average_return"] = price["adjclose"].pct_change(5).rolling(100).mean()
    price["prev_return"] = price["adjclose"].pct_change(5)
    
    # Coefficient of Variation
    price["coev"] = price["adjclose"].rolling(100).std() / price["adjclose"].rolling(100).mean()
    price["return_to_risk"] = price["prev_return"] / price["coev"]
    
    # Bollinger Bands
    price["bollinger"] = ((price["adjclose"].rolling(100).mean() - 2 * price["adjclose"].rolling(100).std()) - price["adjclose"]) / price["adjclose"]

    # RSI (Relative Strength Index)
    delta = price["adjclose"].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    price["rsi"] = 100 - (100 / (1 + rs))

    # MACD (Moving Average Convergence Divergence)
    ema_12 = price["adjclose"].ewm(span=12, adjust=False).mean()
    ema_26 = price["adjclose"].ewm(span=26, adjust=False).mean()
    price["macd"] = ema_12 - ema_26
    price["signal_line"] = price["macd"].ewm(span=9, adjust=False).mean()

    # EMA (Exponential Moving Average)
    price["ema_20"] = price["adjclose"].ewm(span=20, adjust=False).mean()

    # ATR (Average True Range)
    high_low = price["adjhigh"] - price["adjlow"]
    high_close = np.abs(price["adjhigh"] - price["adjclose"].shift())
    low_close = np.abs(price["adjlow"] - price["adjclose"].shift())
    tr = high_low.combine(high_close, np.maximum).combine(low_close, np.maximum)
    price["atr"] = tr.rolling(14).mean()

    # Stochastic Oscillator
    low_14 = price["adjlow"].rolling(window=14).min()
    high_14 = price["adjhigh"].rolling(window=14).max()
    price["stochastic"] = (price["adjclose"] - low_14) / (high_14 - low_14) * 100

    # On-Balance Volume (OBV)
    price["obv"] = (np.sign(price["adjclose"].diff()) * price["volume"]).fillna(0).cumsum()

    # Chaikin Money Flow (CMF)
    money_flow_multiplier = ((price["adjclose"] - price["adjlow"]) - (price["adjhigh"] - price["adjclose"])) / (price["adjhigh"] - price["adjlow"])
    money_flow_volume = money_flow_multiplier * price["volume"]
    price["cmf"] = money_flow_volume.rolling(window=20).sum() / price["volume"].rolling(window=20).sum()

    # Williams %R
    price["williams_r"] = (price["adjhigh"].rolling(14).max() - price["adjclose"]) / (price["adjhigh"].rolling(14).max() - price["adjlow"].rolling(14).min()) * -100

    # Commodity Channel Index (CCI)
    typical_price = (price["adjhigh"] + price["adjlow"] + price["adjclose"]) / 3
    price["cci"] = (typical_price - typical_price.rolling(20).mean()) / (0.015 * typical_price.rolling(20).std())

    # Parabolic SAR (Commented out as it's not supported in this code)
    # price["psar"] = ta.trend.psar(high=price["adjhigh"], low=price["adjlow"], close=price["adjclose"], step=0.02, max_step=0.2)

    # Ichimoku Cloud
    price["tenkan_sen"] = (price["adjhigh"].rolling(9).max() + price["adjlow"].rolling(9).min()) / 2
    price["kijun_sen"] = (price["adjhigh"].rolling(26).max() + price["adjlow"].rolling(26).min()) / 2
    price["senkou_span_a"] = (price["tenkan_sen"] + price["kijun_sen"]) / 2
    price["senkou_span_b"] = (price["adjhigh"].rolling(52).max() + price["adjlow"].rolling(52).min()) / 2
    
    # Force Index
    price["force_index"] = price["adjclose"].diff() * price["volume"]
    
    # Elder's Impulse System
    price["elder_impulse"] = np.where((price["adjclose"] > price["adjclose"].shift()) & (price["volume"] > price["volume"].shift()), 1, 0)
    
    # Keltner Channel
    price["keltner_upper"] = price["adjclose"].rolling(20).mean() + 2 * price["adjclose"].rolling(20).std()
    price["keltner_lower"] = price["adjclose"].rolling(20).mean() - 2 * price["adjclose"].rolling(20).std()

    # Fit ARMA(1,1) model with GARCH(1,1) errors
    returns = price[:int(price.index.size/2)]["adjclose"].pct_change(5).dropna()
    model = arch_model(returns, mean='AR', lags=1, vol='Garch', p=1, q=1)
    garch_fit = model.fit(disp='off')
    forecasts = garch_fit.forecast(horizon=returns.size)
    mean_forecast = forecasts.mean
    variance_forecast = forecasts.variance
    forecasts = [0 for i in range(price.index.size - len(mean_forecast.values[0]))]
    forecasts.extend(mean_forecast.values[0])
    price["garch"] = forecasts
    forecasts = [0 for i in range(price.index.size - len(variance_forecast.values[0]))]
    forecasts.extend(variance_forecast.values[0])
    price["garch_var"] = forecasts
    price["garch_rrr"] = price["garch"] / price["garch_var"]
    
    # Fit ARIMA model
    arima_model = ARIMA(price["adjclose"].dropna(), order=(5, 1, 0))  # Example order (p=5, d=1, q=0)
    arima_fit = arima_model.fit()
    price["arima_forecast"] = arima_fit.predict(start=0, end=len(price)-1, dynamic=False)
    for i in range(10):
        price[str(i)] = price["adjclose"].shift(i)
    price["y"] = price["adjclose"].shift(-5)
    return price


In [14]:
prices_data = []
market = ADatabase("market")
market.connect()
for ticker in tqdm(sp500["ticker"]):
    try:
        price = processor.column_date_processing(market.query("prices",{"ticker":ticker})).sort_values("date")
        price = calculate_technical_indicators(price)
        prices_data.append(price.dropna())
    except Exception as e:
        print(ticker, str(e))
        continue
market.disconnect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 503/503 [00:52<00:00,  9.49it/s]


In [15]:
prices = pd.concat(prices_data).sort_values("date")

In [ ]:
# Train the LSTM model
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(1,1)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(prices[:int(prices.index.size/2)][[str(i) for i in range(10)]].values, prices[:int(prices.index.size/2)][["y"]].values, epochs=15, batch_size=32, validation_split=0.2) 
prices["lstm"] = [x[0] for x in model.predict(prices[[str(i) for i in range(10)]].values)]
prices["lstm"] = (prices["lstm"] - prices["adjclose"]) / prices["adjclose"]

Epoch 1/15
1506/1506 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 2712.5315 - val_loss: 415.6286
Epoch 2/15
1506/1506 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 660.7180 - val_loss: 234.5851
Epoch 3/15
1506/1506 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 497.1364 - val_loss: 521.3746
Epoch 4/15
1506/1506 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 533.7063 - val_loss: 2048.4250
Epoch 5/15
1506/1506 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 687.7758 - val_loss: 307.1242
Epoch 6/15
1506/1506 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 442.7867 - val_loss: 833.1418
Epoch 7/15
1506/1506 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 659.4620 - val_loss: 207.0636
Epoch 8/15
1506/1506 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 621.2260 - val_loss: 374.8257
Epoch 9/15
1506/1506 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 498.7113 - val_loss: 312.5965
Epoch 10/15
1506/1506 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 900.1733 - val_loss: 201.6995
Epoch 11/15
1506/1506 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 422.2271 - val

In [ ]:
factors = [
           "average_return"
            ,"prev_return"
           ,"coev"
           ,"bollinger","rsi","signal_line","ema_20","stochastic","atr","return_to_risk","obv","cmf","williams_r","cci","force_index","elder_impulse"
            ,"keltner_upper","keltner_lower","senkou_span_a","senkou_span_b","garch","garch_var","garch_rrr","arima_forecast"
          ]     

In [ ]:
model = XGBRegressor()
model.fit(prices[:int(prices.index.size/2)][factors].values, prices[:int(prices.index.size/2)][["y"]].values) 
prices["xgb"] = model.predict(prices[factors])
prices["xgb"] = (prices["xgb"] - prices["adjclose"]) / prices["adjclose"]
factors.append("xgb")
factors.append("lstm")

In [ ]:
prices = prices[int(prices.index.size/2):]

In [ ]:
analysis = []   
for factor in tqdm(factors):
    for ascending in [True,False]:
        default_portfolio = {"ticker":"","adjclose":0,"quantity":0,"buy_price":0}
        cash = 100
        portfolio = default_portfolio.copy()
        portfolios = []
        for date in list(prices["date"].unique()):
            try:
                today = prices[prices["date"]==date]
                rec = today.sort_values(factor,ascending=ascending).iloc[0]
                if portfolio["ticker"] == "":
                    portfolio["ticker"] = rec["ticker"]
                    portfolio["adjclose"] = rec["adjclose"]
                    portfolio["quantity"] = cash / rec["adjclose"]
                    portfolio["buy_price"] = rec["adjclose"]
                    portfolio["date"] = date
                    cash = 0
                    portfolios.append(portfolio.copy())
                else:
                    ## update
                    updates = today[today["ticker"]==portfolio["ticker"]].iloc[0]
                    portfolio["adjclose"] = updates["adjclose"]
                    portfolio["date"] = date
                    if rec["ticker"] != portfolio["ticker"]:
                        cash = portfolio["adjclose"] * portfolio["quantity"]
                        portfolio["ticker"] = rec["ticker"]
                        portfolio["adjclose"] = rec["adjclose"]
                        portfolio["quantity"] = cash / rec["adjclose"]
                        portfolio["buy_price"] = rec["adjclose"]
                        portfolio["date"] = date
                        cash = 0
                        portfolios.append(portfolio.copy())
                    else:
                        portfolios.append(portfolio.copy())
            except Exception as e:
                print(str(e))
                continue
        states = pd.DataFrame(portfolios)
        states["pv"] = states["adjclose"] * states["quantity"]
        states["trade"] = states["ticker"] != states["ticker"].shift(-1)
        states["return"] = states["pv"].pct_change(5) * 100
        risk = abs(mean([x for x in states["return"] if x < 0]))
        analysis.append({"factor":factor,"ascending":ascending,"return":states.iloc[-1]["pv"]-100,"risk":risk})

In [ ]:
a = pd.DataFrame(analysis)
a["rr"] = a["return"] / a["risk"]
a.sort_values("rr",ascending=False)

In [ ]:
plt.scatter(a["risk"].values,a["return"].values)
plt.show()